In [46]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import Table, MetaData, Column, Integer

### Build and Establish Connection to DB

In [65]:
authentication = ""
host = ''
database = ''
username = ''
password = '{}'  
conn_string = sa.engine.url.URL(
     "mssql+pyodbc",
     username=username,
     password=password,
     host=host,
     port=1433,
     database=database,
     query={"driver": "ODBC Driver 17 for SQL Server", "authentication": authentication},
 )


<ipython-input-65-ce536614c1ef>:6: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  conn_string = sa.engine.url.URL(


In [66]:
engine = sa.create_engine(conn_string, pool_timeout=60)
connection = engine.connect()

### Read DB Contents

In [150]:
query = """
select schema_name(t.schema_id) as schema_name,
       t.name as table_name,
       t.create_date,
       t.modify_date
from sys.tables t
order by schema_name,
         table_name;
"""

rows = pd.read_sql_query(query, connection)
rows

,schema_name,table_name,create_date,modify_date
0,dbo,BuildVersion,2022-05-15 20:04:00.113,2022-05-15 20:04:01.767
1,dbo,ErrorLog,2022-05-15 20:04:00.113,2022-05-15 20:04:01.767
2,dbo,Rel_Building,2022-05-17 22:49:05.513,2022-05-17 22:49:05.513
3,dbo,Rel_BuyoutAgreements,2022-05-17 22:47:28.627,2022-05-17 22:47:28.627
4,dbo,Rel_Demographics,2022-05-17 22:34:37.987,2022-05-17 22:34:37.987
5,dbo,Rel_Eviction_Notices,2022-05-17 21:59:47.223,2022-05-17 21:59:47.223
6,dbo,Rel_Location,2022-05-17 22:25:35.210,2022-05-17 22:47:28.630
7,dbo,SF_Buyout_Agreements_Raw,2022-05-16 00:27:00.917,2022-05-16 00:27:00.943
8,dbo,SF_Eviction_Notices_Raw,2022-05-16 00:41:20.143,2022-05-16 00:41:20.190
9,dbo,Zip_Atlas_Median_Age_Raw,2022-05-16 00:26:38.500,2022-05-16 00:26:38.530


### Clean Up Sample Schema from DB

In [108]:
#find foreign keys preventing tables from being deleted
statement = """
EXEC sp_fkeys @pktable_name = 'Address',@pktable_owner    = N'SalesLT';
"""

response = pd.read_sql_query(statement, connection)
response

,PKTABLE_QUALIFIER,PKTABLE_OWNER,PKTABLE_NAME,PKCOLUMN_NAME,FKTABLE_QUALIFIER,FKTABLE_OWNER,FKTABLE_NAME,FKCOLUMN_NAME,KEY_SEQ,UPDATE_RULE,DELETE_RULE,FK_NAME,PK_NAME,DEFERRABILITY


In [129]:
#find dependencies preventing tables from being deleted
statement = """
EXEC sp_depends 'SalesLT.ProductModelProductDescription';
"""

response = pd.read_sql_query(statement, connection)
response

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The object 'SalesLT.ProductModelProductDescription' does not exist in database 'mySampleDatabase' or is invalid for this operation. (15009) (SQLExecDirectW)")
[SQL: 
EXEC sp_depends 'SalesLT.ProductModelProductDescription';
]
(Background on this error at: http://sqlalche.me/e/14/f405)

In [127]:
#find remaining schema objects
statement = """
SELECT *
FROM sys.objects WHERE schema_id = SCHEMA_ID('SalesLT');
"""

response = pd.read_sql_query(statement, connection)
response

,name,object_id,principal_id,schema_id,parent_object_id,type,type_desc,create_date,modify_date,is_ms_shipped,is_published,is_schema_published


In [128]:
#drop schema objects one at a time, then drop schema
statement = """
DROP SCHEMA SalesLT;
"""

response = pd.read_sql_query(statement, connection)
response

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [138]:
#drop schema objects one at a time, then drop schema
statement = """
DROP TABLE [dbo].[Rel_Eviction_Notices];
"""

response = pd.read_sql_query(statement, connection)
response

ResourceClosedError: This result object does not return rows. It has been closed automatically.

### Create Normalized Tables

In [139]:
#create eviction notices table
statement = """
CREATE TABLE Rel_Eviction_Notices
(
    eviction_auto_id                                     uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    eviction_id                                          VARCHAR(MAX) NOT NULL,
    file_date                                            DATE NOT NULL,
    non_payment                                          BIT NOT NULL,
    breach                                               BIT NOT NULL,
    nuisance                                             BIT NOT NULL,
    illegal_use                                          BIT NOT NULL,
    failure_to_sign_renewal                              BIT NOT NULL,
    access_denial                                        BIT NOT NULL,
    unapproved_subtenant                                 BIT NOT NULL,
    owner_move_in                                        BIT NOT NULL,
    demolition                                           BIT NOT NULL,
    capital_improvement                                  BIT NOT NULL,
    substantial_rehab                                    BIT NOT NULL,
    ellis_act_withdrawal                                 BIT NOT NULL,
    condo_conversion                                     BIT NOT NULL,
    roommate_same_unit                                   BIT NOT NULL,
    other_cause                                          BIT NOT NULL,
    late_payments                                        BIT NOT NULL,
    lead_remediation                                     BIT NOT NULL,
    development                                          BIT NOT NULL,
    good_samaritan_ends                                  BIT NOT NULL,
    constraints_date                                     DATE
)
"""

response = pd.read_sql_query(statement, connection)
response

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [142]:
#create location table
statement = """
CREATE TABLE Rel_Location
(
    location_id                                          uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    block_level_address                                  VARCHAR NOT NULL,
    zipcode                                              INT NOT NULL,
    current_supervisor_districts                         TINYINT,
    analysis_neighborhoods                               TINYINT,
    city                                                 VARCHAR NOT NULL,
    state                                                VARCHAR NOT NULL,
    supervisor_district                                  TINYINT NOT NULL,
    neighborhoods_analysis_boundaries                    VARCHAR,
    location                                             GEOGRAPHY,
    shape                                                GEOGRAPHY,
    sf_find_neighborhoods                                TINYINT,
    current_police_districts                             TINYINT,
    cbd_bid_gbd_boundaries                               TINYINT,
    central_market_tenderloin_boundary                   TINYINT,
    areas_of_vulnerability                               TINYINT,
    central_market_tenderloin_boundary_polygon           TINYINT,
    fix_it_zones                                         TINYINT,
    neighborhoods                                        TINYINT
)
"""

response = pd.read_sql_query(statement, connection)
response

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [145]:
#create demographics table
statement = """
CREATE TABLE Rel_Demographics
(
    demographic_id                                          uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    median_household_income                                 INT NOT NULL,
    population                                              INT NOT NULL,
    zipcode                                                 INT NOT NULL,
    avg_population_age                                      FLOAT NOT NULL,
    people_per_sq_mile                                      FLOAT NOT NULL
)
"""

response = pd.read_sql_query(statement, connection)
response

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [148]:
#create buyout agreements table
statement = """
CREATE TABLE Rel_BuyoutAgreements
(
    case_number_auto_id                                          uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    case_number                                                  VARCHAR NOT NULL,
    location_id                                                  uniqueidentifier NOT NULL FOREIGN KEY REFERENCES Rel_Location(location_id),
    pre_buyout_disclosure_declaration                            DATE,
    buyout_agreement_date                                        DATE,
    address                                                      VARCHAR NOT NULL,
    buyout_amount                                                INT,
    unknown_amount                                               BIT,
    other_consideration                                          BIT
)
"""

response = pd.read_sql_query(statement, connection)
response

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [149]:
#create building table
statement = """
CREATE TABLE Rel_Building
(
    building_id                                          uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    number_of_tenants                                    TINYINT
)
"""

response = pd.read_sql_query(statement, connection)
response

ResourceClosedError: This result object does not return rows. It has been closed automatically.